# PromptTemplate

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name}，帮我起1个具有{country}特色的{gender}名字")
prompt.format(name="算命大师", country="法国", gender="女孩")


'你是一个算命大师，帮我起1个具有法国特色的女孩名字'